# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional,LSTM
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9351241109339880133
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 357433344
locality {
  bus_id: 1
}
incarnation: 8376758140865212082
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [3]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [4]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [5]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [6]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    x_tk = Tokenizer()
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk
tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [7]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen=length, padding='post')
tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


In [8]:
def tagger(decoder_input_sentence):
    bos = "<BOS> "
    eos = " <EOS>"
    final_target = [bos + text + eos for text in decoder_input_sentence] 
    return final_target

decoder_inputs = tagger(french_sentences)

### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [9]:
def preprocess(x, y, z):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :param y: Label List of sentences with EOS and BOS tags for decoder
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)
    preprocess_d, d_tk = tokenize(z)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
    preprocess_d = pad(preprocess_d)
    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)
    preprocess_d= preprocess_d.reshape(*preprocess_d.shape, 1)
    return preprocess_x, preprocess_y,preprocess_d, x_tk, y_tk,d_tk

preproc_english_sentences, preproc_french_sentences, preproc_french_sentences_decode,english_tokenizer, french_tokenizer,french_tokenizer_decode =\
    preprocess(english_sentences, french_sentences,decoder_inputs)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)
french_vocab_size_decod = len(french_tokenizer_decode.word_index)
max_french_sequence_length_decode= preproc_french_sentences_decode.shape[1]

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("Max French sentence length decoder:", max_french_sequence_length_decode)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)
print("French vocabulary size with BOS and EOS:", french_vocab_size_decod)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
Max French sentence length decoder: 23
English vocabulary size: 199
French vocabulary size: 344
French vocabulary size with BOS and EOS: 346


In [10]:
preproc_french_sentences.shape

(137861, 21, 1)

In [11]:
french_tokenizer_decode.word_index

{'est': 1,
 'bos': 2,
 'eos': 3,
 'en': 4,
 'il': 5,
 'les': 6,
 'mais': 7,
 'et': 8,
 'la': 9,
 'parfois': 10,
 'jamais': 11,
 'le': 12,
 "l'": 13,
 'généralement': 14,
 'moins': 15,
 'aimé': 16,
 'au': 17,
 'fruit': 18,
 'préféré': 19,
 'agréable': 20,
 'froid': 21,
 'son': 22,
 'chaud': 23,
 'de': 24,
 'plus': 25,
 'automne': 26,
 'mois': 27,
 'à': 28,
 'elle': 29,
 'citrons': 30,
 'paris': 31,
 'inde': 32,
 'unis': 33,
 'états': 34,
 'france': 35,
 'jersey': 36,
 'new': 37,
 'chine': 38,
 'pendant': 39,
 'pamplemousse': 40,
 'mon': 41,
 'votre': 42,
 'juin': 43,
 'printemps': 44,
 'janvier': 45,
 'hiver': 46,
 'mars': 47,
 'été': 48,
 'mai': 49,
 'septembre': 50,
 'juillet': 51,
 'avril': 52,
 'novembre': 53,
 'décembre': 54,
 'février': 55,
 'octobre': 56,
 'aime': 57,
 'août': 58,
 'merveilleux': 59,
 'relaxant': 60,
 'doux': 61,
 'humide': 62,
 'notre': 63,
 'californie': 64,
 'sec': 65,
 'leur': 66,
 'occupé': 67,
 'pluvieux': 68,
 'calme': 69,
 'beau': 70,
 'habituellement': 7

In [12]:
english_tokenizer.word_index['the']

5

## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [13]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [1]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    learning_rate = 0.01
    
    input_seq = Input(input_shape[1:])
    rnn_basic = GRU(64, return_sequences=True)(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn_basic)

    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

NameError: name 'tests' is not defined

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [ ]:
embeded_rnn_model.predict(tmp_x[:1])[0]

In [77]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    
    # Config Hyperparameters
    learning_rate = 0.01
    embedding_vec=128
    
    # Config Model
    inputs = Input(shape=input_shape[1:])
#     print(inputs.shape)
    embedding_layer = Embedding(input_dim=english_vocab_size,output_dim=embedding_vec)(inputs)
#     print(embedding_layer.shape)
    hidden_layer = GRU(embedding_vec, return_sequences=True)(embedding_layer)
    
    outputs = TimeDistributed(Dense(french_vocab_size,activation='softmax'))(hidden_layer)
    
    # Create Model from parameters defined above
    model_embed = Model(inputs=inputs, outputs=outputs)
    
    model_embed.compile(loss=sparse_categorical_crossentropy,
                 optimizer=Adam(learning_rate),
                 metrics=['accuracy'])
    
    return model_embed
#output_sequence_lenght=21
tests.test_embed_model(embed_model)

tmp_x = pad(preproc_english_sentences, max_french_sequence_length)


# Train the neural network
embeded_rnn_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
embeded_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
# print(logits_to_text(embeded_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 9s 84us/step - loss: 1.7458 - acc: 0.6241 - val_loss: nan - val_acc: 0.8126
Epoch 2/10
110288/110288 [==============================] - 8s 77us/step - loss: 0.4627 - acc: 0.8565 - val_loss: nan - val_acc: 0.8871
Epoch 3/10
110288/110288 [==============================] - 8s 77us/step - loss: 0.3137 - acc: 0.9007 - val_loss: nan - val_acc: 0.9101
Epoch 4/10
110288/110288 [==============================] - 8s 77us/step - loss: 0.2636 - acc: 0.9150 - val_loss: nan - val_acc: 0.9186
Epoch 5/10
110288/110288 [==============================] - 9s 77us/step - loss: 0.2403 - acc: 0.9212 - val_loss: nan - val_acc: 0.9211
Epoch 6/10
110288/110288 [==============================] - 9s 77us/step - loss: 0.2250 - acc: 0.9248 - val_loss: nan - val_acc: 0.9233
Epoch 7/10
110288/110288 [==============================] - 9s 77us/step - loss: 0.2140 - acc: 0.9279 - val_loss: nan - val_acc: 0.92

In [79]:
print(logits_to_text(embeded_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))
new jamais relaxant parfois relaxant janvier janvier il est chaud décembre été 

new jersey est parfois calme en l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [ ]:
def bd_model(input_shape_bi, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
     # Config Hyperparameters
    learning_rate = 0.01
#     embedding_vec = 64
    
    # Config Model
    inputs_bi = Input(input_shape_bi[1:])
 #   print(inputs_bi.shape)
#     embedding_layer_bi = Embedding(input_dim=english_vocab_size,output_dim=embedding_vec)(inputs_bi)
#    print(embedding_layer_bi.shape)
    bidirection_rnn = Bidirectional(LSTM(64, return_sequences=True))(inputs_bi)
    
    outputs = TimeDistributed(Dense(french_vocab_size,activation='softmax'))(bidirection_rnn)
    
     # Create Model from parameters defined above
    model_bi_rnn = Model(inputs_bi,outputs)
    
    model_bi_rnn.compile(loss=sparse_categorical_crossentropy,
                 optimizer=Adam(learning_rate),
                 metrics=['accuracy'])
    
    return model_bi_rnn


tests.test_bd_model(bd_model)


tmp_x_bi = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x_bi = tmp_x_bi.reshape((-1, preproc_french_sentences.shape[-2], 1))

# TODO: Train and Print prediction(s)


bd_model_rnn = bd_model(
    tmp_x_bi.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
bd_model_rnn.fit(tmp_x_bi, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(bd_model_rnn.predict(tmp_x_bi[:1])[0], french_tokenizer))

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [14]:
import numpy as np

encoder_input_data = np.zeros(
  (len(preproc_english_sentences), max_english_sequence_length),
  dtype='float32')
decoder_input_data = np.zeros(
  (len(preproc_english_sentences), max_french_sequence_length_decode,1),
  dtype='float32')
decoder_target_data = np.zeros(
  (len(preproc_english_sentences), max_french_sequence_length_decode,1),
  dtype='float32')

In [15]:
for i, (input_text, target_text) in enumerate(zip(preproc_english_sentences, preproc_french_sentences_decode)):
    for t, word in enumerate(input_text):
            encoder_input_data[i, t] = word
    for t, char in enumerate(target_text):
        s=char[0]
        decoder_input_data[i, t,0]=s
        if t > 0:
      # decoder_target_data will be ahead by one timestep and will not include the start character.
          decoder_target_data[i, t - 1,0] = s

In [16]:
encoder_input_data = encoder_input_data.reshape((-1, preproc_english_sentences.shape[-1], 1))
decoder_input_data = decoder_input_data.reshape((-1, preproc_french_sentences_decode.shape[-2], 1)) 
decoder_target_data = decoder_target_data.reshape((-1, preproc_french_sentences_decode.shape[-2], 1))

In [39]:
encoder_input_data.shape

(137861, 15, 1)

In [21]:
decoder_target_data[:1]

array([[[  37.],
        [  36.],
        [   1.],
        [  10.],
        [  69.],
        [  39.],
        [  13.],
        [  26.],
        [   8.],
        [   5.],
        [   1.],
        [ 114.],
        [   4.],
        [  52.],
        [   3.],
        [   0.],
        [   0.],
        [   0.],
        [   0.],
        [   0.],
        [   0.],
        [   0.],
        [   0.]]], dtype=float32)

In [22]:
dec

[37, 36, 1, 10, 23, 39, 13, 26, 8, 5, 1, 114, 4, 52, 3, 0, 0, 0, 0, 0, 0, 0]

In [86]:
decoder_input_data[44,22,0]

0.0

In [17]:
def encdec_model(input_shape, output_shape, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
   # TODO: Implement

    n_units = 256
    
    learning_rate =  0.01

    encoder_inputs = Input(input_shape[1:])

    encoder_lstm = LSTM(n_units, return_state=True)

    encoder_output, state_h, state_c = encoder_lstm(encoder_inputs)

    encoder_states = [state_h, state_c]

  #  decoder_inputs = RepeatVector(23)(encoder_output)
    
    decoder_inputs= Input(output_shape[1:])

    decoder_lstm  = LSTM(n_units, return_sequences=True, return_state=True)

    decoder_output, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

    decoder_dense = TimeDistributed(Dense(french_vocab_size, activation='softmax'))

    decoder_outputs = decoder_dense(decoder_output)

    endec_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    endec_model.compile(loss=sparse_categorical_crossentropy,optimizer=Adam(learning_rate),metrics=['accuracy'])
    #define inference encoder
    encoder_model = Model(encoder_inputs, encoder_states)
    # define inference decoder
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
    
 
    return endec_model,encoder_model,decoder_model






# # OPTIONAL: Train and Print prediction(s)

train_model,encoder_model,decoder_model = encdec_model(encoder_input_data.shape,decoder_input_data.shape,english_vocab_size,french_vocab_size_decod)
train_model.fit([encoder_input_data, decoder_input_data],decoder_target_data, batch_size=1024, epochs=100, validation_split=0.2)

Train on 110288 samples, validate on 27573 samples
Epoch 1/100
110288/110288 [==============================] - 21s 193us/step - loss: 1.1281 - acc: 0.6952 - val_loss: nan - val_acc: 0.7692
Epoch 2/100
110288/110288 [==============================] - 19s 174us/step - loss: 0.6664 - acc: 0.7688 - val_loss: nan - val_acc: 0.7854
Epoch 3/100
110288/110288 [==============================] - 19s 175us/step - loss: 0.5432 - acc: 0.8021 - val_loss: nan - val_acc: 0.8118
Epoch 4/100
110288/110288 [==============================] - 19s 176us/step - loss: 0.4682 - acc: 0.8215 - val_loss: nan - val_acc: 0.8358
Epoch 5/100
110288/110288 [==============================] - 19s 175us/step - loss: 0.4123 - acc: 0.8366 - val_loss: nan - val_acc: 0.8345
Epoch 6/100
110288/110288 [==============================] - 19s 175us/step - loss: 0.3775 - acc: 0.8463 - val_loss: nan - val_acc: 0.8523
Epoch 7/100
110288/110288 [==============================] - 19s 175us/step - loss: 0.3677 - acc: 0.8500 - val_loss

In [113]:
preproc_french_sentences_decode.shape[-2]

23

In [114]:
    target_seq = np.zeros(decoder_input_data.shape[1:])
    target_seq=target_seq.reshape((-1, preproc_french_sentences_decode.shape[-2], 1))

In [115]:
target_seq.shape

(1, 23, 1)

In [116]:
french_tokenizer_decode.word_index['new']

37

In [18]:
def predict_sequence(infenc, infdec, source,output_shape):
    #encode the source sequence
    state = infenc.predict(source)
    target_seq = np.zeros(output_shape[1:])
    target_seq=target_seq.reshape((-1, preproc_french_sentences_decode.shape[-2], 1))
    print(target_seq.shape)
    target_seq[0,0,0]=2.0
    # collect predictions
    stop_cond= False   
    decode_sent=[]
    k=0
    while(stop_cond==False):
        output_word, h, c = infdec.predict([target_seq] + state)        
    #         print('sdf',logits_to_text(output_word,french_tokenizer_decode))
        output_word=output_word.reshape((23,346))
        decode_sent.append(np.argmax(output_word[k]))
        target_seq[0,k+1,0]=np.argmax(output_word[k])
        if len(decode_sent)>21:
            stop_cond=True
        k+=1
    return decode_sent



In [19]:
# Print prediction(s)
# return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])
dec=predict_sequence(encoder_model,decoder_model,encoder_input_data[:1],decoder_input_data.shape)

(1, 23, 1)


In [25]:
index_to_words = {id: word for word, id in french_tokenizer_decode.word_index.items()}
index_to_words[0] = '<PAD>'

print( ' '.join([index_to_words[prediction] for prediction in dec]))


new jersey est parfois chaud pendant l' automne et il est neigeux en avril eos <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [26]:
import numpy as np

encoder_input_data_final = np.zeros(
  (len(preproc_english_sentences), max_english_sequence_length),
  dtype='float32')
decoder_input_data_final = np.zeros(
  (len(preproc_english_sentences), max_french_sequence_length_decode,1),
  dtype='float32')
decoder_target_data_final = np.zeros(
  (len(preproc_english_sentences), max_french_sequence_length_decode,1),
  dtype='float32')

for i, (input_text, target_text) in enumerate(zip(preproc_english_sentences, preproc_french_sentences_decode)):
    for t, word in enumerate(input_text):
            encoder_input_data_final[i, t] = word
    for t, char in enumerate(target_text):
        s=char[0]
        decoder_input_data_final[i, t,0]=s
        if t > 0:
      # decoder_target_data_final will be ahead by one timestep and will not include the start character.
          decoder_target_data_final[i, t - 1,0] = s

In [ ]:
def final_model(input_shape, output_shape, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
   # TODO: Implement

    n_units = 256
    
    learning_rate =  0.01
    embedding_vec=128


    encoder_inputs = Input(input_shape[1:])

    embedding_layer = Embedding(input_dim=english_vocab_size,output_dim=embedding_vec)(encoder_inputs)

    encoder_lstm = LSTM(n_units, return_state=True)

    encoder_output, state_h, state_c = encoder_lstm(embedding_layer)

    encoder_states = [state_h, state_c]

  #  decoder_inputs = RepeatVector(23)(encoder_output)
    
    decoder_inputs= Input(output_shape[1:])

    decoder_lstm  = LSTM(n_units, return_sequences=True, return_state=True)

    decoder_output, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

    decoder_dense = TimeDistributed(Dense(french_vocab_size, activation='softmax'))

    decoder_outputs = decoder_dense(decoder_output)

    endec_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    endec_model.compile(loss=sparse_categorical_crossentropy,optimizer=Adam(learning_rate),metrics=['accuracy'])
    #define inference encoder
    encoder_model_final = Model(encoder_inputs, encoder_states)
    # define inference decoder
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model_final = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
    
 
    return endec_model,encoder_model_final,decoder_model_final






# # OPTIONAL: Train and Print prediction(s)

final_train,encoder_model_final,decoder_model_final = final_model(encoder_input_data_final.shape,decoder_input_data_final.shape,english_vocab_size,french_vocab_size_decod)
final_train.fit([encoder_input_data_final, decoder_input_data_final],decoder_target_data_final, batch_size=1024, epochs=10, validation_split=0.2)

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 22s 197us/step - loss: 1.1365 - acc: 0.7029 - val_loss: nan - val_acc: 0.8046
Epoch 2/10
110288/110288 [==============================] - 21s 190us/step - loss: 0.4626 - acc: 0.8441 - val_loss: nan - val_acc: 0.8854
Epoch 3/10
110288/110288 [==============================] - 21s 190us/step - loss: 0.2162 - acc: 0.9301 - val_loss: nan - val_acc: 0.9689
Epoch 4/10
110288/110288 [==============================] - 21s 189us/step - loss: 0.0765 - acc: 0.9798 - val_loss: nan - val_acc: 0.9858
Epoch 5/10
110288/110288 [==============================] - 21s 190us/step - loss: 0.0437 - acc: 0.9882 - val_loss: nan - val_acc: 0.9892
Epoch 6/10
109568/110288 [============================>.] - ETA: 0s - loss: 0.0315 - acc: 0.9915

## Prediction (IMPLEMENTATION)

In [ ]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    model = None

    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

### Generate the html

**Save your notebook before running the next cell to generate the HTML output.** Then submit your project.

In [ ]:
# Save before you run this cell!
!!jupyter nbconvert *.ipynb

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?